In [1]:
!pip install selenium

DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [2]:
!pip install beautifulsoup4

DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [3]:
!pip install webdriver-manager

DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import time

import re
import pandas as pd

In [5]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [6]:
def web_driver():
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless')
    options.add_argument('--window-size=1920, 1200')
    options.add_argument('--lang=pt-BR')
    # options.add_argument('--lang=en-US')
    
    driver = webdriver.Chrome(
        service=ChromeService(ChromeDriverManager().install()),
        options=options
    )
    
    return driver

# web_driver()

def open_page(url):
    driver = web_driver()
    driver.get(url)
    soup = get_page_source(driver)
    return driver, soup
    
def get_page_source(driver):
    res = driver.page_source.encode('utf-8')
    soup = BeautifulSoup(res, 'html.parser')
    return soup

In [10]:
driver, soup = open_page('https://store.steampowered.com/charts/mostplayed')

time.sleep(2)

soup = get_page_source(driver)

tb_rank = soup.find('table', attrs={'class': '_3arZn0BMPzyhcYNADe193m'})
tbody_rank = tb_rank.find('tbody')
tr_rank = tbody_rank.find_all('tr')

#print(tr_rank)

In [11]:
def find_non_none(tag1, tag2):
    if tag1:
        print('tentei2')
        return tag1
    elif tag2:
        print('tentei')
        return tag2
    return None

In [12]:
ls_jogos = []

for jogo_listado in tr_rank:
    caminho = jogo_listado.find('td', attrs={'class': '_18kGHKeOavDDdJVs9FVhpo'})
    nome = caminho.find('a').find('div').text
    jogadores_atuais = jogo_listado.find('td', attrs={'class': '_3L0CDDIUaOKTGfqdpqmjcy'}).text
    jogadores_atuais = jogadores_atuais.replace('.','')
    pico_jogadores = jogo_listado.find('td', attrs={'class': 'yJB7DYKsuTG2AYhJdWTIk'}).text
    pico_jogadores = pico_jogadores.replace('.','')
    caminho = jogo_listado.find('td', attrs={'class': '_3IyfUchPbsYMEaGjJU3GOP'})
    preco = ''
    try:
        preco = caminho.find('div').find('div').find('div', attrs={'class': '_3NhLu7mTdty7JufpSpz6Re'}).find('div', attrs={'class': '_3j4dI1yA7cRfCvK8h406OB'}).text
        preco = preco.replace('R$','')
        preco = preco.replace(',','.')
    except Exception as e:
        #print(e)
        try:
            preco = caminho.find('div').find('div').find('div').text
            preco = preco.replace('Gratuito para jogar', '0')
            preco = preco.replace('Em breve','0')
            preco = preco.replace('Novidade','0')
            preco = preco.replace('R$','')
            preco = preco.replace(',','.')
        except:
            preco = '0'
    
    ls_jogos.append({
        'Nome': nome,
        'Preço': preco,
        'Jogadores Atuais': jogadores_atuais,
        'Pico de Jogadores': pico_jogadores
    })

In [13]:
df = pd.DataFrame.from_dict(ls_jogos)

df

,Nome,Preço,Jogadores Atuais,Pico de Jogadores
0,Counter-Strike 2,0,641099,1330043
1,Banana,0,449233,485762
2,Dota 2,0,341497,624244
3,Call of Duty®,0,220985,291093
4,THRONE AND LIBERTY,0,123960,147977
5,PUBG: BATTLEGROUNDS,0,106093,670563
6,Rust,103.49,75518,105612
7,Factorio,101.99,75455,92674
8,Grand Theft Auto V,55.05,68085,127932
9,Baldur's Gate 3,159.99,64138,86805


In [14]:
df.dtypes

Nome                 object
Preço                object
Jogadores Atuais     object
Pico de Jogadores    object
dtype: object

In [15]:
import pickle

with open('df_jogos.pkl', 'wb') as pkl_file:
    pickle.dump(df, pkl_file)